In [1]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from datetime import date

# Exract

In [2]:
#set pathways to read csv 
us_vacc_path = "resources/vaccinations/us_state_vaccinations.csv"
state_business_path = "resources/State.csv"
mobility_path= "resources/Global_Mobility_Report.csv"
us_path = "resources/2021_US_Region_Mobility_Report.csv"
unemployment_rate_path = "resources/unemployment_rate_2019-2021.csv"

#convert csv into dataframe
vacc_df = pd.read_csv(us_vacc_path)
state_business_df = pd.read_csv(state_business_path)
us_mobility = pd.read_csv(us_path)
unemployment_df = pd.read_csv(unemployment_rate_path)

# Transform

### Vaccine Data 

In [3]:
vacc_df = vacc_df[["date", "location", "people_vaccinated", "people_fully_vaccinated"]]
vacc_df = vacc_df.dropna(axis="index")
vacc_df = vacc_df.loc[vacc_df["date"] >= "2019-01-01"]
vacc_df = vacc_df.rename(columns={"location":"state"})
vacc_df

,date,state,people_vaccinated,people_fully_vaccinated
0,2021-01-12,Alabama,70861.0,7270.0
1,2021-01-13,Alabama,74792.0,9245.0
3,2021-01-15,Alabama,86956.0,13488.0
7,2021-01-19,Alabama,114319.0,16346.0
8,2021-01-20,Alabama,121113.0,17956.0
...,...,...,...,...
2813,2021-02-19,Wyoming,87289.0,36651.0
2814,2021-02-20,Wyoming,89346.0,39323.0
2815,2021-02-21,Wyoming,89605.0,39474.0
2816,2021-02-22,Wyoming,91931.0,43289.0


In [4]:
vacc_df["state"].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'Bureau of Prisons', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Dept of Defense', 'District of Columbia',
       'Federated States of Micronesia', 'Florida', 'Georgia', 'Guam',
       'Hawaii', 'Idaho', 'Illinois', 'Indian Health Svc', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Long Term Care', 'Louisiana',
       'Maine', 'Marshall Islands', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota',
       'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Republic of Palau', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
       'United States', 'Utah', 'Vermont', 'Veterans Health',
       'Virgin Islands', 'Virginia', 'Washington', 'West V

In [5]:
vacc_df = vacc_df.set_index("state")

In [6]:
vacc_df = vacc_df.drop(index=["American Samoa","Bureau of Prisons", "Dept of Defense", "District of Columbia", "Federated States of Micronesia", "Guam",
                   "Indian Health Svc", "Long Term Care", "Marshall Islands","Northern Mariana Islands", "Puerto Rico",
                    "Republic of Palau", "Veterans Health"], axis=0)

vacc_df = vacc_df.reset_index()

vacc_df

,state,date,people_vaccinated,people_fully_vaccinated
0,Alabama,2021-01-12,70861.0,7270.0
1,Alabama,2021-01-13,74792.0,9245.0
2,Alabama,2021-01-15,86956.0,13488.0
3,Alabama,2021-01-19,114319.0,16346.0
4,Alabama,2021-01-20,121113.0,17956.0
...,...,...,...,...
1959,Wyoming,2021-02-19,87289.0,36651.0
1960,Wyoming,2021-02-20,89346.0,39323.0
1961,Wyoming,2021-02-21,89605.0,39474.0
1962,Wyoming,2021-02-22,91931.0,43289.0


## Business Data

In [7]:
state_business_df = state_business_df[["Year", "Week", "State", "Business Applications", "Applications from Corporations"]]
state_business_df

,Year,Week,State,Business Applications,Applications from Corporations
0,2006,1,AK,90,30
1,2006,2,AK,90,30
2,2006,3,AK,130,40
3,2006,4,AK,100,20
4,2006,5,AK,150,30
...,...,...,...,...,...
40744,2021,13,WY,670,90
40745,2021,14,WY,700,100
40746,2021,15,WY,760,110
40747,2021,16,WY,680,100


In [8]:
#Change abbreviation 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

state_business_df = state_business_df.replace({"State": abbrev_us_state})
state_business_df = state_business_df.loc[state_business_df["Year"] >= 2019]
state_business_df

,Year,Week,State,Business Applications,Applications from Corporations
678,2019,1,Alaska,120,10
679,2019,2,Alaska,220,20
680,2019,3,Alaska,180,10
681,2019,4,Alaska,160,20
682,2019,5,Alaska,230,20
...,...,...,...,...,...
40744,2021,13,Wyoming,670,90
40745,2021,14,Wyoming,700,100
40746,2021,15,Wyoming,760,110
40747,2021,16,Wyoming,680,100


In [9]:
state_business_df = state_business_df.groupby(["Year"]).mean()
state_business_df = state_business_df.drop(columns={"Week"})
state_business_df = state_business_df.reset_index()
state_business_df

,Year,Business Applications,Applications from Corporations
0,2019,1319.306184,209.479638
1,2020,1637.349170,213.744344
2,2021,2393.829296,268.915802


## United States Mobility

In [10]:
us_mobility = us_mobility[us_mobility['sub_region_2'].isnull()]
us_mobility = us_mobility[us_mobility['sub_region_1'].notnull()]

In [11]:
us_mobility = us_mobility[["sub_region_1", "date", "retail_and_recreation_percent_change_from_baseline",
                          "grocery_and_pharmacy_percent_change_from_baseline", "parks_percent_change_from_baseline",
                          "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline",
                          "residential_percent_change_from_baseline"]]
us_mobility = us_mobility.dropna()

us_mobility = us_mobility.rename(columns={"sub_region_1":"state","retail_and_recreation_percent_change_from_baseline":"retail_and_rec_percent_change",
                                "grocery_and_pharmacy_percent_change_from_baseline":"grocery_and_pharmacy_percent_change",
                                "parks_percent_change_from_baseline":"parks_percent_change","transit_stations_percent_change_from_baseline":
                                "transit_stations_percent_change", "workplaces_percent_change_from_baseline":"workplace_percent_change",
                                "residential_percent_change_from_baseline":"residential_percent_change"})
us_mobility

,state,date,retail_and_rec_percent_change,grocery_and_pharmacy_percent_change,parks_percent_change,transit_stations_percent_change,workplace_percent_change,residential_percent_change
157,Alabama,2021-01-01,-44.0,-30.0,-16.0,-46.0,-72.0,26.0
158,Alabama,2021-01-02,-19.0,-8.0,-2.0,-8.0,-19.0,7.0
159,Alabama,2021-01-03,-16.0,-8.0,-7.0,-1.0,-17.0,6.0
160,Alabama,2021-01-04,-5.0,2.0,-9.0,-9.0,-26.0,8.0
161,Alabama,2021-01-05,-10.0,-1.0,-14.0,-12.0,-25.0,8.0
...,...,...,...,...,...,...,...,...
405753,Wyoming,2021-06-02,16.0,22.0,235.0,30.0,-24.0,0.0
405754,Wyoming,2021-06-03,19.0,27.0,240.0,43.0,-24.0,0.0
405755,Wyoming,2021-06-04,8.0,22.0,220.0,55.0,-23.0,1.0
405756,Wyoming,2021-06-05,9.0,21.0,248.0,57.0,-11.0,-3.0


## United States Unemployment Data

In [12]:
unemployment_df

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2019,4.0,3.8,3.8,3.7,3.7,3.6,3.6,3.7,3.5,3.6,3.6,3.6
1,2020,3.5,3.5,4.4,14.8,13.3,11.1,10.2,8.4,7.8,6.9,6.7,6.7
2,2021,6.3,6.2,6.0,6.1,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge Data

In [13]:
merge_df = pd.merge(vacc_df, us_mobility)
merge_df

,state,date,people_vaccinated,people_fully_vaccinated,retail_and_rec_percent_change,grocery_and_pharmacy_percent_change,parks_percent_change,transit_stations_percent_change,workplace_percent_change,residential_percent_change
0,Alabama,2021-01-12,70861.0,7270.0,-15.0,-7.0,-25.0,-15.0,-24.0,9.0
1,Alabama,2021-01-13,74792.0,9245.0,-13.0,-6.0,-18.0,-14.0,-24.0,8.0
2,Alabama,2021-01-15,86956.0,13488.0,-18.0,-12.0,-16.0,-17.0,-24.0,9.0
3,Alabama,2021-01-19,114319.0,16346.0,-19.0,-9.0,-18.0,-16.0,-23.0,8.0
4,Alabama,2021-01-20,121113.0,17956.0,-16.0,-10.0,-10.0,-15.0,-24.0,8.0
...,...,...,...,...,...,...,...,...,...,...
1842,Wyoming,2021-02-19,87289.0,36651.0,-19.0,-10.0,-17.0,-10.0,-12.0,6.0
1843,Wyoming,2021-02-20,89346.0,39323.0,-15.0,-10.0,-4.0,-6.0,-9.0,4.0
1844,Wyoming,2021-02-21,89605.0,39474.0,-14.0,-12.0,-20.0,-17.0,-11.0,4.0
1845,Wyoming,2021-02-22,91931.0,43289.0,-14.0,-6.0,-4.0,-15.0,-16.0,5.0


In [18]:
unemp_business_merge = pd.merge(unemployment_df, state_business_df, how="outer")
unemp_business_merge

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Business Applications,Applications from Corporations
0,2019,4.0,3.8,3.8,3.7,3.7,3.6,3.6,3.7,3.5,3.6,3.6,3.6,1319.306184,209.479638
1,2020,3.5,3.5,4.4,14.8,13.3,11.1,10.2,8.4,7.8,6.9,6.7,6.7,1637.349170,213.744344
2,2021,6.3,6.2,6.0,6.1,5.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2393.829296,268.915802


# Load

In [15]:
#connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [65]:
#check for tables
engine.table_names()

['vacc_mobility_2021', 'unemployment_rate']

In [24]:
merge_df.to_sql(name="vacc_mobility_2021", con=engine, if_exists="append", index=False)

In [26]:
unemployment_df.to_sql(name="unemployment_rate", con=engine, if_exists="append", index=False)